# Subsamples and neighborhoods

In this example we go through the ability of Connectome Utilities to generate control subpopulations.
That is, imagine you have a large network, and a smaller sub-network of it, based on a subset of the nodes and all connections between them. You may find that the subnetwork appears to be non-random with respect to some analysis.
At that point there are two options: The non-random aspect can be explained in terms of a non-random sampling from nodes with a given property, or not.

As an example: In cortical neuronal networks we know that there are generally two types of nodes: Excitatory or Inhibitory. The first tends to have a larger in-degree, the second a larger out-degree. If your sample is biased towards one of the two groups, this may already explain your non-random aspect.

To test this, we can generate random control subnetworks that match the subnetwork under investigation with respect to a node property. Here, we present an illustrative example.

We begin by loading data. In this example, we load neuron-to-neuron connectivity of a few thousand neurons in a biologically detailed model. It also loads a list of "properties" associated with the nodes (neurons), such as their locations in two different coordinate systems.

We demonstrate how the data would be loaded from a circuit model in Sonata format, but in case you don't have a Sonata circuit available, there is also a serialized version provided with this repository.

In [1]:
import numpy, pandas
import bluepysnap as snap
import conntility
import os

from scipy.spatial import distance
from matplotlib import pyplot as plt


# Describes which node properties to load and filtering based on them
# This loads a small subvolume of aorund 6.5k nodes
load_cfg = {
    "loading":{    
        "properties": ["x", "y", "z", "mtype", "layer", "synapse_class"],
        "atlas": [
            {
                "data": "./data/ss_fm.nrrd", 
                "properties": ["ss_flat_x", "depth", "ss_flat_y"]
            }
        ],
        "base_target": "hex_O1",
        "node_population": "S1nonbarrel_neurons"
    },
    "filtering":[
        {
            "column": "synapse_class",
            "values": ["EXC"]
        },
        {
            "column": "layer",
            "values": [2, 3]
        },
        {
            "column": "ss_flat_x",
            "interval": [1200, 1600]
        },
        {
            "column": "ss_flat_y",
            "interval": [2300, 2700]
        }
    ]
}

# Some additional configuration
fn_circ = "./data/circuit_config_multipop.json"
fn_con_mat = "./data/L23_EXC_sampled_cmat.h5"
population = 'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical'

# Since you may not have access to the Sonata circuit this example is based on, we provide the result also in hdf5
if os.path.exists(fn_con_mat):
    cmat = conntility.ConnectivityMatrix.from_h5(fn_con_mat)
else:
    circ = snap.Circuit(fn_circ)
    cmat = conntility.ConnectivityMatrix.from_bluepy(circ, load_config=load_cfg, population=population,
                                                    connectome="S1nonbarrel_neurons__S1nonbarrel_neurons__chemical")
    cmat.to_h5(fn_con_mat)

100%|██████████| 6439/6439 [00:22<00:00, 281.30it/s]


Using the .index operator, we first generate a non-random subpopulation. In this example, we first sample all nodes where the value of the "ss_flat_x" property (its x-coordinate) is between 1350 and 1400. Then we randomly pick 100 nodes from the result. As in this dataset connectivity is strongly distance-dependent, this will result in a connection probability higher than in the overall network.

The first sampling is done by chaining a "lt" (less than) indexing operation with a "gt" (greater than) operation. The second is done by randomly picking from the "gid" node property, then accessing the corresponding .subpopulation

In [2]:
subpop = cmat.index("ss_flat_x").lt(1400).index("ss_flat_x").gt(1350)
subpop = subpop.subpopulation(numpy.random.choice(subpop.vertices["node_ids"], 100, replace=False))

Confirm that the connection probability is higher than in the base population.

In [3]:
print("""
The subpopulation has a connection probability of {0}%,
compared to that, the overall network has a connection probability of {1}%""".format(
    100 * subpop.matrix.mean(),
    100 * cmat.matrix.mean()))


The subpopulation has a connection probability of 3.1900000000000004%,
compared to that, the overall network has a connection probability of 1.7219888674873722%


As a test, we generate a control subpopulation with the same distribution of the "mtype" property as our subpopulation. We confirm that it matches by considering the difference.

In [4]:
rnd_control_mtype = cmat.index("mtype").random_categorical(subpop.gids)

display(subpop.vertices["mtype"].value_counts() - rnd_control_mtype.vertices["mtype"].value_counts())

mtype
L3_TPC:A     0
L2_TPC:B     0
L3_TPC:C     0
L2_TPC:A     0
L2_IPC       0
L5_DBC       0
L5_LBC       0
L5_MC        0
L5_NBC       0
L5_NGC       0
L5_SBC       0
L5_TPC:A     0
L5_TPC:B     0
L5_TPC:C     0
L5_UPC       0
L6_BP        0
L1_DAC       0
L5_BTC       0
L6_BPC       0
L6_BTC       0
L6_CHC       0
L6_DBC       0
L6_HPC       0
L6_IPC       0
L6_LBC       0
L6_MC        0
L6_NBC       0
L6_NGC       0
L6_SBC       0
L6_TPC:A     0
L6_TPC:C     0
L5_CHC       0
L4_TPC       0
L5_BP        0
L23_NGC      0
L1_LAC       0
L1_NGC-DA    0
L1_NGC-SA    0
L1_SAC       0
L23_BP       0
L23_BTC      0
L23_CHC      0
L23_DBC      0
L23_LBC      0
L23_MC       0
L23_NBC      0
L23_SBC      0
L4_UPC       0
L4_BP        0
L4_BTC       0
L4_CHC       0
L4_DBC       0
L4_LBC       0
L4_MC        0
L4_NBC       0
L4_NGC       0
L4_SBC       0
L4_SSC       0
L1_HAC       0
L6_UPC       0
Name: count, dtype: int64

This random control sample does not explain the increased connection probability, as expected.

In [5]:
print("""
The subpopulation has a connection probability of {0}%,
compared to that, the random control has a connection probability of {1}%""".format(
    100 * subpop.matrix.mean(),
    100 * rnd_control_mtype.matrix.mean()))


The subpopulation has a connection probability of 3.1900000000000004%,
compared to that, the random control has a connection probability of 1.8599999999999999%


When generating a random control according to a numerical property, it is first binned. The number of bins used can be provided.
We confirm that the control matches the sample when the same number of bins is used.
However, with respect to a larger number of bins it does not match, as expected.

In [6]:
rnd_control_depth = cmat.index("depth").random_numerical(subpop.gids, n_bins=25)

H_subpop, bins = numpy.histogram(subpop.vertices["depth"], bins=25)
H_control_depth = numpy.histogram(rnd_control_depth.vertices["depth"], bins=bins)[0]

print(H_subpop - H_control_depth)

H_subpop, bins = numpy.histogram(subpop.vertices["depth"], bins=50)
H_control_depth = numpy.histogram(rnd_control_depth.vertices["depth"], bins=bins)[0]

print(H_subpop - H_control_depth)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0  0 -2  2  0  0  0  0  0  0 -1  1  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  1 -1  0  0  0  0  0  0  0  0  0  0  2 -2  0  0
  1 -1]


Still, no explanation of the increased connection probability.

In [7]:
print("""
The subpopulation has a connection probability of {0}%,
compared to that, the random control has a connection probability of {1}%""".format(
    100 * subpop.matrix.mean(),
    100 * rnd_control_depth.matrix.mean()))


The subpopulation has a connection probability of 3.1900000000000004%,
compared to that, the random control has a connection probability of 1.53%


Now we generate a control with respect to "ss_flat_x". As this is the property that first defined the non-random subpopulation, we expect this to have approximately the same connection probability.

In [8]:
rnd_control_x = cmat.index("ss_flat_x").random_numerical(subpop.gids, n_bins=25)

print("""
The subpopulation has a connection probability of {0}%,
compared to that, the random control has a connection probability of {1}%""".format(
    100 * subpop.matrix.mean(),
    100 * rnd_control_x.matrix.mean()))


The subpopulation has a connection probability of 3.1900000000000004%,
compared to that, the random control has a connection probability of 3.020000000000001%


## Neighborhood sampling

Another way of generating subpopulations is using the very connectivity of the network: Neighborhood sampling.
The neighborhood of a node is the set of nodes connected to it. We can easily sample the neighborhood of a node by providing the "gids" property of the node.

By default, the "center" of the neighborhood, i.e. the node that is connected to all others, is placed first in the resulting subpopulation.
We confirm that the first node is connected to all others.

In [9]:
from matplotlib import pyplot as plt

neighborhood = cmat.neighborhood[subpop.gids[0]]
print(neighborhood.array[0] | neighborhood.array[:, 0])

[False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

We have more control if we use the .get method instead of indexing. For example, we can set center_first to False, which keeps the order of the nodes in the base population instead of placing the center at index 0.

By providing a "gid" to the "pre=" kwarg, we consider only the _efferently_ connected neighborhood. Similarly, we could use the "post=" kwarg for the _afferent_ neighborhood.

We confirm that the result has one node with an out-degree equal to the size of the sample minus one.

In [10]:
neighborhood = cmat.neighborhood.get(pre=subpop.gids[0], center_first=False)

print(neighborhood.array.sum(axis=1) / (len(neighborhood) - 1))

[0.02061856 0.01030928 0.15463918 0.08247423 0.04123711 0.01030928
 0.07216495 0.09278351 0.03092784 1.         0.01030928 0.01030928
 0.01030928 0.08247423 0.02061856 0.05154639 0.02061856 0.05154639
 0.12371134 0.02061856 0.04123711 0.02061856 0.06185567 0.08247423
 0.05154639 0.03092784 0.05154639 0.03092784 0.04123711 0.05154639
 0.02061856 0.04123711 0.04123711 0.07216495 0.08247423 0.04123711
 0.11340206 0.08247423 0.02061856 0.09278351 0.         0.02061856
 0.04123711 0.         0.04123711 0.10309278 0.01030928 0.05154639
 0.04123711 0.03092784 0.04123711 0.04123711 0.05154639 0.09278351
 0.05154639 0.         0.03092784 0.05154639 0.02061856 0.01030928
 0.07216495 0.02061856 0.03092784 0.         0.06185567 0.
 0.06185567 0.06185567 0.05154639 0.08247423 0.04123711 0.08247423
 0.02061856 0.02061856 0.01030928 0.06185567 0.09278351 0.05154639
 0.         0.07216495 0.04123711 0.02061856 0.01030928 0.02061856
 0.         0.01030928 0.         0.06185567 0.01030928 0.02061856
 0.

If we provide more than one "gid" as input, the result will be "ConnectivityGroup", i.e. a group of subpopulations. It can be indexed by the "gids" in the input. Additionally, it provides an .index property.

In [11]:
grp = cmat.neighborhood[subpop.gids[:10]]

smpl = grp[subpop.gids[0]]
print(smpl.array[0] | smpl.array[:, 0])  # Confirm the "center" is connected to all others

print(grp.index)

[False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

Here is something we can do as an example: 
Is the distribtion of the "mtype" property of the sampled afferent neighborhoods non-random with respect to the base population?
That is, do the nodes in a sample receive inputs from a non-random populations with respect to the "mtype" property? Or do they sample their inputs randomly from the base population?

In [12]:
grp = cmat.neighborhood.get(post=subpop.gids[:10])

neighborhood_mtypes = pandas.concat([grp[_gid].vertices["mtype"].value_counts(normalize=True)
                                     for _gid in grp.index], axis=1)

diff_df = neighborhood_mtypes.mean(axis=1) - cmat.vertices["mtype"].value_counts(normalize=True)
zscore_df = diff_df.divide(neighborhood_mtypes.std(axis=1))

display(zscore_df)

mtype
L1_DAC            NaN
L1_HAC            NaN
L1_LAC            NaN
L1_NGC-DA         NaN
L1_NGC-SA         NaN
L1_SAC            NaN
L23_BP            NaN
L23_BTC           NaN
L23_CHC           NaN
L23_DBC           NaN
L23_LBC           NaN
L23_MC            NaN
L23_NBC           NaN
L23_NGC           NaN
L23_SBC           NaN
L2_IPC       0.246065
L2_TPC:A    -0.609844
L2_TPC:B     0.554598
L3_TPC:A    -0.400533
L3_TPC:C    -0.346611
L4_BP             NaN
L4_BTC            NaN
L4_CHC            NaN
L4_DBC            NaN
L4_LBC            NaN
L4_MC             NaN
L4_NBC            NaN
L4_NGC            NaN
L4_SBC            NaN
L4_SSC            NaN
L4_TPC            NaN
L4_UPC            NaN
L5_BP             NaN
L5_BTC            NaN
L5_CHC            NaN
L5_DBC            NaN
L5_LBC            NaN
L5_MC             NaN
L5_NBC            NaN
L5_NGC            NaN
L5_SBC            NaN
L5_TPC:A          NaN
L5_TPC:B          NaN
L5_TPC:C          NaN
L5_UPC            NaN
L6_B